In [1]:
# SECTION 1: Setup and Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Flatten, Dropout
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.applications import Xception, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.optimizers import Adam
import time

# Set paths
REAL_PATH = '/content/drive/MyDrive/EE656/real'
FAKE_PATH = '/content/drive/MyDrive/EE656/fake'

# SECTION 2: Frame Extraction and Preprocessing (with Haar face detection)
def extract_faces_from_video(video_path, max_frames=10):  # reduced for memory efficiency
    faces = []
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces_rect = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces_rect:
            face = frame[y:y+h, x:x+w]
            face = cv2.resize(face, (299, 299))
            faces.append(face)
            break  # only one face per frame
        frame_count += 1
    cap.release()
    return faces

# Load real and fake frames (limit dataset size for memory)
def load_dataset(max_videos_per_class=10):
    X, y, vids = [], [], []
    for label, folder in enumerate([REAL_PATH, FAKE_PATH]):
        for i, filename in enumerate(tqdm(os.listdir(folder))):
            if filename.endswith('.mp4'):
                if i >= max_videos_per_class:
                    break
                video_path = os.path.join(folder, filename)
                faces = extract_faces_from_video(video_path)
                for face in faces:
                    X.append(face / 255.0)
                    y.append(label)
                    vids.append(filename)
    return np.array(X), np.array(y), np.array(vids)

X, y, vids = load_dataset()

# SECTION 3: Train/Test Split
X_train, X_test, y_train, y_test, vids_train, vids_test = train_test_split(X, y, vids, test_size=0.2, stratify=y, random_state=42)
#Resize for MesoNet
X_train_m = np.array([cv2.resize((img*255).astype(np.uint8), (256,256))/255.0 for img in X_train])
X_test_m  = np.array([cv2.resize((img*255).astype(np.uint8), (256,256))/255.0 for img in X_test])

# SECTION 4: Build Models
# Xception
inp = Input(shape=(299, 299, 3))
xception = Xception(include_top=False, weights='imagenet', input_tensor=inp)
x = GlobalAveragePooling2D()(xception.output)
x = Dense(1, activation='sigmoid')(x)
model_x = Model(inputs=inp, outputs=x)
model_x.compile(optimizer=Adam(learning_rate=2e-4), loss='binary_crossentropy', metrics=['AUC'])

# Patch-ResNet (Layer1)
inp2 = Input(shape=(299, 299, 3))
resnet = ResNet50(include_top=False, weights='imagenet', input_tensor=inp2)
res_layer1 = resnet.get_layer('conv2_block3_out').output
y = GlobalAveragePooling2D()(res_layer1)
y = Dense(1, activation='sigmoid')(y)
model_r = Model(inputs=inp2, outputs=y)
model_r.compile(optimizer=Adam(learning_rate=2e-4), loss='binary_crossentropy', metrics=['AUC'])

# --- EfficientNetB0 ---
inp3 = Input(shape=(299, 299, 3))
effnet = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=inp3)
e = GlobalAveragePooling2D()(effnet.output)
e = Dense(1, activation='sigmoid')(e)
model_e = Model(inputs=inp3, outputs=e)
model_e.compile(optimizer=Adam(learning_rate=2e-4),
                loss='binary_crossentropy', metrics=['AUC'])

# --- MesoNet (256×256 input) ---
def build_mesonet(input_shape=(256,256,3)):
    m = Sequential()
    m.add(Conv2D(8, (3, 3), padding='same', input_shape=input_shape))
    m.add(BatchNormalization()); m.add(Activation('relu'))
    m.add(Conv2D(8, (5, 5), strides=2, padding='same'))
    m.add(BatchNormalization()); m.add(Activation('relu'))
    m.add(Conv2D(16, (3, 3), padding='same'))
    m.add(BatchNormalization()); m.add(Activation('relu'))
    m.add(Conv2D(16, (5, 5), strides=2, padding='same'))
    m.add(BatchNormalization()); m.add(Activation('relu'))
    m.add(Flatten())
    m.add(Dropout(0.5))
    m.add(Dense(1, activation='sigmoid'))
    m.compile(optimizer=Adam(learning_rate=2e-4),
              loss='binary_crossentropy', metrics=['AUC'])
    return m

# Instantiate MesoNet
model_meso = build_mesonet()

# SECTION 5: Training (reduce batch size and epochs)
model_x.fit(X_train, y_train, epochs=2, batch_size=16, validation_split=0.1)
model_r.fit(X_train, y_train, epochs=2, batch_size=16, validation_split=0.1)
model_e.fit(X_train,   y_train, epochs=2, batch_size=16, validation_split=0.1)
model_meso.fit(X_train_m, y_train, epochs=2, batch_size=16, validation_split=0.1)

# SECTION 6: Evaluation
pred_x = model_x.predict(X_test)
pred_r = model_r.predict(X_test)
auc_frame_x = roc_auc_score(y_test, pred_x)
auc_frame_r = roc_auc_score(y_test, pred_r)
print("Frame-level AUC - Xception:", auc_frame_x)
print("Frame-level AUC - Patch-ResNet:", auc_frame_r)
# SECTION 6.1: Evaluate EfficientNetB0
pred_e = model_e.predict(X_test).flatten()
auc_e = roc_auc_score(y_test, pred_e)
acc_e = accuracy_score(y_test, (pred_e>0.5).astype(int))
print("Frame-level AUC - EfficientNetB0:", auc_e)
print("Frame-level Accuracy - EfficientNetB0:", acc_e)

# SECTION 6.2: Evaluate MesoNet
pred_meso = model_meso.predict(X_test_m).flatten()
auc_m = roc_auc_score(y_test, pred_meso)
acc_m = accuracy_score(y_test, (pred_meso>0.5).astype(int))
print("Frame-level AUC - MesoNet:", auc_m)
print("Frame-level Accuracy - MesoNet:", acc_m)

# Video-level AUC
video_df = pd.DataFrame({
    'video': vids_test,
    'label': y_test,
    'pred_x': pred_x.flatten(),
    'pred_r': pred_r.flatten()
})
vid_preds_x = video_df.groupby('video')['pred_x'].mean()
vid_labels = video_df.groupby('video')['label'].first()
vid_preds_r = video_df.groupby('video')['pred_r'].mean()
auc_vid_x = roc_auc_score(vid_labels, vid_preds_x)
auc_vid_r = roc_auc_score(vid_labels, vid_preds_r)
print("Video-level AUC - Xception:", auc_vid_x)
print("Video-level AUC - Patch-ResNet:", auc_vid_r)

# SECTION 7: Params and Inference Time
print("Xception params:", model_x.count_params())
print("Patch-ResNet params:", model_r.count_params())

start = time.time()
_ = model_x.predict(np.expand_dims(X_test[0], axis=0))
print("Xception time:", (time.time() - start) * 1000, "ms")

start = time.time()
_ = model_r.predict(np.expand_dims(X_test[0], axis=0))
print("Patch-ResNet time:", (time.time() - start) * 1000, "ms")

# SECTION 7.1: Params & Inference Time
print("EfficientNetB0 params:", model_e.count_params())
start = time.time(); _ = model_e.predict(np.expand_dims(X_test[0],axis=0))
print("EfficientNetB0 time:", (time.time()-start)*1000, "ms")

print("MesoNet params:", model_meso.count_params())
start = time.time(); _ = model_meso.predict(np.expand_dims(X_test_m[0],axis=0))
print("MesoNet time:", (time.time()-start)*1000, "ms")

# SECTION 8: Save Model and Results
model_x.save('/content/drive/MyDrive/xception_model.h5')
model_r.save('/content/drive/MyDrive/patchresnet_model.h5')

Mounted at /content/drive


 20%|██        | 10/49 [00:09<00:37,  1.05it/s]


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2
9/9 ━━━━━━━━━━━━━━━━━━━━ 276s 26s/step - AUC: 0.8583 - loss: 0.4243 - val_AUC: 1.0000 - val_loss: 1.4743
Epoch 2/2
9/9 ━━━━━━━━━━━━━━━━━━━━ 275s 28s/step - AUC: 1.0000 - loss: 0.0324 - val_AUC: 1.0000 - val_loss: 1.7022
Epoch 1/2
9/9 ━━━━━━━━━━━━━━━━━━━━ 82s 8s/step - AUC: 0.7018 - loss: 0.7371 - val_AUC: 0.5000 - val_loss: 2.8939
Epoch 2/2
9/9 ━━━━━━━━━━━━━━━━━━━━ 80s 7s/step - AUC: 0.9173 - loss: 0.4623 - val_AUC: 0.5000 - val_loss: 3.0215
Epoch 1/2
9/9 ━━━━━━━━━━━━━━━━━━━━ 145s 9s/step - AUC: 0.8605 - loss: 0.5445 - val_AUC: 0.6000 - val_loss: 0.7256
Epoch 2/2
9/9 ━━━━━━━━━━━━━━━━━━━━ 137s 9s/step - AUC: 1.0000 - loss: 0.1369 - val_AUC: 0.5000 - val_loss: 0.7200
Epoch 1/2
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - AUC: 0.6482 - loss: 0.7401 - val_AUC: 0.6400 - val_loss: 0.7123
Epoch 2/2
9/9 ━━━━━━━━━━━━━━━━━━━━ 18s 882ms/step - AUC: 0.9619 - loss: 0.2448 - val_AUC: 0.8100 - val_loss: 0.7062
2/2 ━━━━━━━━━━━━━━━━━━━━ 20s 5s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 951ms/step
Frame-level

1/2 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step
Frame-level AUC - EfficientNetB0: 0.6305555555555555
Frame-level Accuracy - EfficientNetB0: 0.47368421052631576
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 517ms/step
Frame-level AUC - MesoNet: 0.7805555555555556
Frame-level Accuracy - MesoNet: 0.47368421052631576
Video-level AUC - Xception: 1.0
Video-level AUC - Patch-ResNet: 0.8624999999999999
Xception params: 20863529
Patch-ResNet params: 230017
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step
Xception time: 638.8368606567383 ms
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Patch-ResNet time: 162.89782524108887 ms
EfficientNetB0 params: 4050852
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
EfficientNetB0 time: 221.93288803100586 ms
MesoNet params: 75145
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


MesoNet time: 120.45764923095703 ms
